In [ ]:
import cv2
import easyocr
import openpyxl

In [ ]:
# creating excel sheet 
wb = openpyxl.Workbook()
ws = wb.active
ws.title = "Number Plates"
ws['A1'] = 'Image Name'
ws['B1'] = 'Number Plate Text'

In [ ]:
vid = cv2.VideoCapture(0)

count = 0

img_path = os.path.join('Number_Plates/scanned_images')

while True:
    ret, frame = vid.read()
    gray_image = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    cascasde = cv2.CascadeClassifier('haarcascade_russian_plate_number.xml')
    
    number_plates = cascasde.detectMultiScale(gray_image, 1.1, 4)
    
    for (x,y,w,h) in number_plates:
        cv2.rectangle(frame, (x,y), (x+w,y+h), (255,0,0), 2)
        cv2.putText(frame, "Number Plate", (x,y-5), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (255, 0, 255), 2)
        
        plate_area = frame[y:y+h, x:x+w]
        cv2.imshow('Number Plate', plate_area)
    
    
        reader = easyocr.Reader(['en'])
        number = reader.readtext(plate_area, detail=0)
    
        if number:
            number_text = number[0]
            print("Detected Number:", number_text)
            
            img_name = 'Number_Plates/scanned_images' + str(count) + '.jpg'
            ws.append([img_name, number_text])
            count += 1
    
    cv2.imshow('Number Plate', frame)
    
    if cv2.waitKey(1) & 0XFF == ord('s'):
        cv2.imwrite(img_name, plate_area)
        cv2.rectangle(frame, (0,200), (640,300), (0,255,0), cv2.FILLED)
        cv2.putText(frame, "Plate Saved", (150, 265), cv2.FONT_HERSHEY_COMPLEX_SMALL, 2, (0, 0, 255), 2)
        cv2.imshow("Results",frame)
        cv2.waitKey(500)
        
    if cv2.waitKey(1) & 0XFF ==ord('x'):
        break

wb.save('number_plates.xlsx')        
wb.close()
        